In [1]:
#In this notebook, we estimate the time constants T_d and T_p using linear regression by taking time estimations E[T] of many points with different N and K values

In [80]:
import numpy as np
import random
import time
import math

In [3]:
def CollisionCalc(X0, X1, R):
    return abs(X0-X1) < (2 * R)

def TotalCollisions(Points, R):
    calc=  0
    for i in range(len(Points)):
        j = i+1
        while j < len(Points):
            calc+=CollisionCalc(Points[i], Points[j], R)
            j+=1

    return calc

In [4]:
def generateSample(N, K, Samples=1000):
    Dataset = []
    DatasetK = np.random.random((Samples, K))
    for x in range(Samples):
        Partitions = [[] for x in range(N)]
        #add values to partitions
        for i in range(K):
            Partitions[int((DatasetK[x][i]*N)-(1e-5))].append(DatasetK[x][i])

        Dataset.append(Partitions)
    return Dataset

In [5]:
def SampleTest(N, K, Dataset, R=0.004):
    Cols = []
    for dat in Dataset:
        Cols.append(0)
        for i in range(N-1):
            if len(dat[i]) == 0:
                continue
            Cols[-1]+=TotalCollisions(dat[i]+dat[i+1], R)
    return Cols

In [6]:
def EstimateTimeSample(N, K, R=0.004, Samples=1000):
    DataSet = generateSample(N, K, Samples=Samples)
    tstart = time.time()
    Ans = SampleTest(N, K, DataSet, R = R)
    return (time.time()-tstart)/Samples

In [7]:
def DataSet(Nlow, Nhigh, Nstep, Klow, Khigh, Kstep, Samples = 20):
    timeSteps = []
    for i in range(Samples):
        N = random.randint(int(Nlow/Nstep), int(Nhigh/Nstep))*Nstep
        K = random.randint(int(Klow/Kstep), int(Khigh/Kstep))*Kstep
        timeSteps.append([N, K, EstimateTimeSample(N, K, R=0.004, Samples=100)])
    return timeSteps

In [37]:
def DataSetPrep(Dataset):
    #change the (N, K, E[T]) fields in dataset to (N, 2K(K-1)(N-1)/N^2, E[T])
    Dat = []
    for x in range(len(Dataset)):
        n = Dataset[x][0]
        k = Dataset[x][1]
        t = Dataset[x][2]
        Dat.append([n, 2*k*(k-1)*(n-1)/(n*n), t])
    return Dat

In [70]:
X = DataSet(100, 3000, 100, 20, 300, 10, Samples=200)

In [87]:
X = np.array(X)
print(X[0])

[2.00000000e+03 2.50000000e+02 1.00063086e-03]


In [72]:
Y = DataSetPrep(X)

In [73]:
def SampleDatasetToLinearRegForm(Dataset):
    X = []
    Y = []
    for d in Dataset:
        X.append([d[0], d[1], 1])
        Y.append(d[2])
    return (np.array(X), np.array(Y))

In [74]:
Z = SampleDatasetToLinearRegForm(Y)

In [75]:
def linearRegression(X, Y):
    Vars = np.matmul(np.linalg.inv(np.matmul(X.transpose(), X)), np.matmul(X.transpose(), Y))
    return Vars

In [76]:
T_p, T_d, C = linearRegression(Z[0], Z[1])

In [77]:
print(T_p, T_d, C)

1.8603147966921272e-07 1.0537108325436089e-06 0.00019701972469200356


In [99]:
ratio = T_d/T_p
Value = np.sqrt(2 * X[:,1] * (X[:,1]-1) * (X[:,0]-1) * ratio)

In [103]:
print(Value-X[:,0])

[35545.55901969 37528.29131314  9325.02244551 19952.54411658
  5104.32203246  4908.76005979 28241.97970221  4908.76005979
 47921.13315608 32317.7778581   3477.35901744 14186.27160121
  8886.13894124 36877.73091293  9617.54730526 35681.09316877
 15651.24728011 13242.85980184 40926.82624624  2461.10405151
  1222.70317646 30086.20027359 10221.79283679  8886.13894124
 21266.10366083 24037.04781194  5329.46314118 28241.97970221
 11465.84763369 10568.94049521 42805.87405858 16265.765254
 24497.09687256 10146.7418323   1675.3025208  44423.9801503
 14698.0886576  16277.27836924 18180.0532141  39178.24888704
  5061.77214196 44423.9801503   8255.44866331  2436.63728475
 37915.22611743  9452.27272907 13137.24567592 20223.00029837
 13177.76670514  2255.40792489 31066.58538002 33705.05390965
 36285.65075198 27942.01604825 31012.69682451  8153.98302885
 44423.9801503  18832.34945705 17942.4882622  17052.62682486
 25894.1044307  30861.15135256  2462.47353179 16943.40513312
 16265.765254   30943.61845

In [105]:
#conclusion: In 1-dimensional case, the most optimized version of the grid is where the grid size is equal to the ball size radius 0.004
#This is true for most grid sizes unless the balls are very small in size